In [1]:
import numpy as np
from scipy import linalg
import numpy as np
from scipy import linalg
import pandas as pd
import warnings
warnings.simplefilter('ignore', DeprecationWarning)

data = pd.read_csv("http://www.ats.ucla.edu/stat/data/binary.csv")
data.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [2]:
X=data.get(["gre","gpa","rank"])
Y=data.get(["admit"])
nx, mx = X.shape

beta = np.zeros((mx, 1))
epsilon = 1e-6
beta

array([[ 0.],
       [ 0.],
       [ 0.]])

In [3]:
def exp_it(_x):
    x = _x.copy()
    y = 1 / (1 + np.exp(-x))
    return y


In [4]:
def mylm_qr(A):
    n, m = A.shape
    R = A.copy()
    Q = np.eye(n)

    for k in range(m-1):
        x = np.zeros((n, 1))
        x[k:, 0] = R[k:, k]
        v = x
        v[k] = x[k] + np.sign(x[k,0]) * np.linalg.norm(x)

        s = np.linalg.norm(v)
        if s != 0:
            u = v / s
            R -= 2 * np.dot(u, np.dot(u.T, R))
            Q -= 2 * np.dot(u, np.dot(u.T, Q))
    Q = Q.T
    R1 = R[:m-1, :m-1]
    Y1 = R[:m-1, m-1]
    beta = np.linalg.solve(R1, Y1)
    beta = pd.DataFrame(beta)#make the beta a dataframe , this will sovle the problem that when y-pr the data type is diffent 
    return beta

In [7]:
def mylogistic(_x, _y):
    x = _x.copy()
    y = _y.copy()
    r, c = x.shape

    beta = np.zeros((c, 1))
    epsilon = 1e-6

    while True:
        eta = np.dot(x, beta)
        pr = exp_it(eta)
        w = pr * (1 - pr)
        z = eta + (y - pr) / w
        sw = np.sqrt(w)
        mw = np.repeat(sw, c, axis=1)

        x_work = mw * x
        y_work = sw * z
        rw,cw=x_work.shape
        A = pd.concat([x_work, y_work], axis=1)
        a=A.as_matrix(columns=None)

        beta_new=mylm_qr(a)
        #beta_new, _, _, _ = np.linalg.lstsq(x_work, y_work)
        
        err = np.sum(np.abs(beta_new - beta))
        beta = beta_new
        #beta here is a dataframe so the comparison must be specific 
        if err[0] < epsilon:
            break

    return beta

In [8]:
if __name__ == '__main__':
#     n = 1000
#     p = 5

#     X = np.random.normal(0, 1, (n, p))
#     #beta = np.arange(p) + 1
#     beta = np.ones((p, 1))
#     print beta

#     Y = np.random.uniform(0, 1, (n, 1)) < exp_it(np.dot(X, beta)).reshape((n, 1))
    A = pd.concat([X, Y], axis=1)
    logistic_beta = mylogistic(X, Y)
    print logistic_beta

          0
0  0.001477
1 -0.004167
2 -0.669538


In [9]:
#The Result without intercept
#           0
# 0  0.001477
# 1 -0.004167
# 2 -0.669538

In [10]:
intercept =pd.DataFrame(np.ones((nx, 1)))
A = pd.concat([intercept, X], axis=1)
logistic_beta = mylogistic(A, Y)
print logistic_beta

          0
0 -3.449548
1  0.002294
2  0.777014
3 -0.560031


In [11]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(X,Y)

/Users/Dongzhe/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, penalty='l2',
          random_state=None, tol=0.0001)

In [12]:
#compare the result with builtin logistic regression model 
print(round(logreg.intercept_[0],6),round(logreg.coef_[0][0],6),round(logreg.coef_[0][1],6),round(logreg.coef_[0][2],6))

(-3.418297, 0.002275, 0.770528, -0.55881)
